<a href="https://colab.research.google.com/github/Nisha0344/Python_yolov7-project/blob/main/object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
!git clone https://github.com/WongKinYiu/yolov7.git

Cloning into 'yolov7'...
remote: Enumerating objects: 1197, done.
remote: Total 1197 (delta 0), reused 0 (delta 0), pack-reused 1197 (from 1)
Receiving objects: 100% (1197/1197), 74.23 MiB | 21.92 MiB/s, done.
Resolving deltas: 100% (520/520), done.


In [8]:
%cd yolov7

[Errno 2] No such file or directory: 'yolov7'
/content/yolov7


In [9]:
pip install -r requirements.txt

In [10]:
import cv2
import numpy as np
from pathlib import Path

video_path = "yolov7/data/video/input_video.mp4"
Path("yolov7/data/video").mkdir(parents=True, exist_ok=True)

width, height = 640, 480
fps = 30
duration = 5
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(video_path, fourcc, fps, (width, height))

for i in range(fps * duration):
    frame = np.zeros((height, width, 3), dtype=np.uint8)
    cv2.rectangle(frame, (50 + i, 100), (150 + i, 200), (0, 255, 0), -1)
    out.write(frame)

out.release()
print(f"Sample video created at {video_path}")


Sample video created at yolov7/data/video/input_video.mp4


In [11]:
import cv2
import numpy as np
import torch
import json
import time
from pathlib import Path
import matplotlib.pyplot as plt


In [13]:
model = torch.hub.load('WongKinYiu/yolov7', 'yolov7')

/usr/local/lib/python3.10/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/WongKinYiu/yolov7/zipball/main" to /root/.cache/torch/hub/main.zip
/usr/local/lib/python3.10/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.

100%|██████████| 72.1M/72.1M [00:00<00:00, 97.5MB/s]
/root/.cache/torch/hub/WongKinYiu_yolov7_main/hubconf.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.


Adding autoShape... 


In [17]:
import json
from pathlib import Path

def create_hierarchical_json(detections, class_names):
    hierarchy = []
    for det in detections:
        x1, y1, x2, y2, conf, cls_id = det
        obj = {
            "object": class_names[int(cls_id)],
            "id": f"object_{int(cls_id)}",
            "bbox": [int(x1), int(y1), int(x2), int(y2)],
            "confidence": round(float(conf), 2),
            "subobjects": []
        }

        if int(cls_id) in [0, 1]:
            obj["subobjects"].append({
                "subobject": "example_subobject",
                "id": f"subobject_{int(cls_id)}_sub",
                "bbox": [int(x1) + 5, int(y1) + 5, int(x2) - 5, int(y2) - 5],
                "confidence": round(float(conf) * 0.9, 2)
            })

        hierarchy.append(obj)

    return hierarchy

detections = [
    [50, 50, 150, 150, 0.9, 0],
    [200, 200, 300, 300, 0.8, 1]
]

class_names = ['person', 'car', 'truck', 'helmet', 'tire', 'door']

hierarchical_json = create_hierarchical_json(detections, class_names)

output_path = Path("/content/detection_output.json")
with output_path.open("w") as f:
    json.dump(hierarchical_json, f, indent=4)

print(f"Hierarchical JSON saved to {output_path}")


Hierarchical JSON saved to /content/detection_output.json


In [18]:
def generate_json(detections):
    return [{
        'x1': int(det[0]),
        'y1': int(det[1]),
        'x2': int(det[2]),
        'y2': int(det[3]),
        'confidence': float(det[4]),
        'class_id': int(det[5]),
    } for det in detections]

In [19]:
video_path = "yolov5/data/video/traffic.mp4"
Path("yolov5/data/video").mkdir(parents=True, exist_ok=True)

width, height = 640, 480
fps = 30
duration = 5
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(video_path, fourcc, fps, (width, height))

for i in range(fps * duration):
    frame = np.zeros((height, width, 3), dtype=np.uint8)
    cv2.rectangle(frame, (50 + i, 100), (150 + i, 200), (0, 255, 0), -1)
    out.write(frame)

out.release()
print(f"Sample video created at {video_path}")


Sample video created at yolov5/data/video/traffic.mp4


In [20]:
import cv2
import numpy as np
import torch
import json
import time
from pathlib import Path
import matplotlib.pyplot as plt

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Unable to open video file {video_path}")
        returnall_detections = []
    frames_output = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break


        results = model(frame)
        detections = results.xyxy[0].cpu().numpy()

        all_detections.extend(generate_json(detections))


        for det in detections:
            x1, y1, x2, y2, conf, cls = det
            label = f"Class_{int(cls)}: {conf:.2f}"
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


        frames_output.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    cap.release()
    save_json(all_detections)
    return frames_output

In [21]:
import time
start_time = time.time()
end_time = time.time()
fps = 1 / (end_time - start_time)
print(f"Inference Speed: {fps} FPS")

Inference Speed: 32017.58778625954 FPS
